In [2]:
import json,re,os
import pandas as pd
import openai
from IPython.display import display, Markdown

In [3]:
TRAINING_DATA_DIR = "labeled_data"
data = []
for dir in os.listdir(TRAINING_DATA_DIR):
    prompt = open(f'{TRAINING_DATA_DIR}/{dir}/prompt.md','r').read().strip()
    code = open(f'{TRAINING_DATA_DIR}/{dir}/code.md','r').read()
    data.append({'prompt':prompt, 'code':code})


In [4]:
# training_df = pd.read_json("training_data_fixed.jsonl", lines=True)
training_df = pd.json_normalize(data)
print(training_df.info())
training_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   prompt  20 non-null     object
 1   code    20 non-null     object
dtypes: object(2)
memory usage: 452.0+ bytes
None


,prompt,code
0,You are an AI Teacher guiding a student toward...,```python\n\n# Define the goal and concise spe...
1,You are an AI Teacher guiding a student toward...,```python\n# Adding recursion to the root of t...
2,You are an AI Teacher guiding a student toward...,```python\n# Creating a customized learning pl...
3,You are an AI Teacher guiding a student toward...,```python\n# Updating memory to mark the stude...
4,You are an AI Teacher guiding a student toward...,"```python\nquestion = """"""### Problem: Longest ..."


In [5]:
def etract_components(prompt):
    pattern = r'''(.*)# Tools Available:(.*)# Memory:(.*)# Additional Context Gathered:(.*)# Conversation History \(Last .+\):(.*)# Additional Info:(.*)# Learning Plan:(.*)# Current Time:(.*)# Instructions:(.*)# Here is the last input from student.(.*)# Plan code:'''

    match = re.search(pattern, prompt, re.DOTALL)

    if match:
        preamble = match.group(1).strip()
        tools_available = match.group(2).strip()
        memory = match.group(3).strip()
        additional_context = match.group(4).strip()
        conversation_history = match.group(5).strip()
        additional_info = match.group(6).strip()
        learning_plan = match.group(7).strip()
        current_time = match.group(8).strip()
        instructions = match.group(9).strip()
        student_input = match.group(10).strip()
        # Create a dictionary with the extracted values
        data = {
            "preamble": preamble,
            "Tools Available": tools_available,
            "Memory": memory,
            "Additional Context Gathered": additional_context,
            "Conversation History (Last 10)": conversation_history,
            "Additional Info": additional_info,
            "Learning Plan": learning_plan,
            "Current Time": current_time,
            "Instructions": instructions,
            "Student Input": student_input
        }

        # Print the dictionary
        return data

    else:
        print(match)
        raise ValueError("regex not matching :")
# print(training_df.prompt.iloc[0])
etract_components(training_df.prompt.iloc[0])

{'preamble': 'You are an AI Teacher guiding a student toward their goals. Prepare customized plans, track their progress and skills, and personalize your responses using humor, challenges, and appreciation based on their personality ,goal and progreass.',
 'Tools Available': "- 'MemoryManager':\n\n    A class to manage in-memory short key-value pairs along with their update timestamps.\n    \n    \n    * `get_memory(key: str) -> str` \t# Retrieves the value associated with a given key from memory.\n    \n    * `get_memory_snapshot() -> str` \t# Retrieves a formatted snapshot of the memory, showing keys, values and the time elapsed since their last update.\n    \n    * `update_memory(key: str, value: str) -> None` \t# Updates the memory with a new short key-value pair. Make sure values are less than 200 characters.\n\n- 'MCQExaminer':\n\n    * `get_question(subject: str, difficulty: str) -> str` \t# Get an MCQ question based on the subject and difficulty level.\n    \n    * `grade_answe

In [6]:
prompt_components_df = pd.json_normalize(training_df.prompt.apply(etract_components))
components_df  = pd.concat([training_df, prompt_components_df], axis = 1)
components_df.head()

,prompt,code,preamble,Tools Available,Memory,Additional Context Gathered,Conversation History (Last 10),Additional Info,Learning Plan,Current Time,Instructions,Student Input
0,You are an AI Teacher guiding a student toward...,```python\n\n# Define the goal and concise spe...,You are an AI Teacher guiding a student toward...,- 'MemoryManager':\n\n A class to manage in...,,,- *student*: Hello\n\n- *Tutor*: Hello there! ...,None,No plan locked yet.,2024-09-03 19:06:23.208722,- Generate Only Python Code to call the approp...,Student: im intrested in algorithms and maybe...
1,You are an AI Teacher guiding a student toward...,```python\n# Adding recursion to the root of t...,You are an AI Teacher guiding a student toward...,- 'MemoryManager':\n\n A class to manage in...,,,- *student*: Hello\n\n- *Tutor*: Hello there! ...,None,"- **Learning Plan** (Importance: 1, Score: 0.0...",2024-09-03 19:07:12.670582,- Generate Only Python Code to call the approp...,Student: can you help me improve my recursion?
2,You are an AI Teacher guiding a student toward...,```python\n# Creating a customized learning pl...,You are an AI Teacher guiding a student toward...,- 'MemoryManager':\n\n A class to manage in...,,,- *student*: Hello\n\n- *Tutor*: Hello there! ...,None,No plan locked yet.,2024-09-03 17:37:52.274059,- Generate Only Python Code to call the approp...,Student: Coding interview at Google
3,You are an AI Teacher guiding a student toward...,```python\n# Updating memory to mark the stude...,You are an AI Teacher guiding a student toward...,- 'MemoryManager':\n\n A class to manage in...,learning_goal: Ds algo (updated 21m 11s ago)\n...,,"- *student*: Yeah, sure. Arrays and strings s...",None,"- **Learning Plan** (Importance: 1, Score: 0.0...",2024-09-03 17:17:06.594050,- Generate Only Python Code to call the approp...,Student: Seriously? What's the task already? J...
4,You are an AI Teacher guiding a student toward...,"```python\nquestion = """"""### Problem: Longest ...",You are an AI Teacher guiding a student toward...,- 'MemoryManager':\n\n A class to manage in...,learning_goal: Ds algo (updated 30m 35s ago)\n...,,"- *student*: Yeah, that sounds good. Show me h...",None,"- **Learning Plan** (Importance: 1, Score: 0.0...",2024-09-03 17:26:30.220272,- Generate Only Python Code to call the approp...,Student: I dont know how to write it


In [8]:
index = 5
display(Markdown(components_df['Student Input'].iloc[index]))
display(Markdown(components_df['code'].iloc[index]))

Student: This looks simple.

```python
# Updating the memory with the student's preference for array and string difficulty.
self.tools['MemoryManager'].update_memory(key="preference_arrays_and_strings_difficulty", value="intermediate")

# Retrieve an intermediate-level coding question on array manipulation.
advanced_array_question = self.tools['CodingPracticeTutor'].get_question(
    topic="array manipulation in Python",
    difficulty="intermediate"
)

# Respond with the intermediate-level question and offer hints if needed.
self.tools['Responder'].respond(
    f"Since you have a basic understanding of array manipulation, let's challenge you with an intermediate-level question.\
\
{advanced_array_question}\
\
Can you solve this? Let me know if you need hints."
)
```

# Evaluation metric

1. fraction of Correct Method calls - jaccard distance
2. Edit distance

In [9]:
def list_levenshtein_distance(list1, list2):
    """
    Calculate the Levenshtein distance between two lists of words.

    :param list1: First list of words.
    :param list2: Second list of words.
    :return: The Levenshtein distance between the two lists.
    """
    m, n = len(list1), len(list2)
    dp = [[0] * (n + 1) for _ in range(m + 1)]

    # Initialize the distance matrix
    for i in range(m + 1):
        dp[i][0] = i
    for j in range(n + 1):
        dp[0][j] = j

    # Compute the distance matrix
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            cost = 0 if list1[i - 1] == list2[j - 1] else 1
            dp[i][j] = min(dp[i - 1][j] + 1,    # Deletion
                           dp[i][j - 1] + 1,    # Insertion
                           dp[i - 1][j - 1] + cost)  # Substitution

    return dp[m][n]

def normalized_list_levenshtein_distance(list1, list2):
    """
    Calculate the normalized Levenshtein distance between two lists of words.

    The normalized Levenshtein distance is the raw Levenshtein distance divided
    by the maximum length of the two lists, resulting in a value between 0 and 1.

    :param list1: First list of words.
    :param list2: Second list of words.
    :return: The normalized Levenshtein distance between the two lists.
    """
    # Compute the Levenshtein distance
    lev_distance = list_levenshtein_distance(list1, list2)
    
    # Find the maximum length of the two lists
    max_length = max(len(list1), len(list2))
    
    # Normalize the distance
    normalized_distance = lev_distance / max_length if max_length > 0 else 0
    
    return normalized_distance

# Example usage
list1 = ['apple', 'banana', 'cherry']
list2 = ['apple', 'banana', 'banana']

print(normalized_list_levenshtein_distance(list1, list2))  # Output: edit distance between the two lists

def jaccard_similarity(list1, list2):
    """
    Calculate the Jaccard Similarity between two lists of words.

    :param list1: First list of words.
    :param list2: Second list of words.
    :return: The Jaccard Similarity between the two lists.
    """
    set1 = set(list1)
    set2 = set(list2)
    
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    
    return intersection / union if union != 0 else 0

def jaccard_distance(list1, list2):
    """
    Calculate the Jaccard Distance between two lists of words.

    :param list1: First list of words.
    :param list2: Second list of words.
    :return: The Jaccard Distance between the two lists.
    """
    return 1 - jaccard_similarity(list1, list2)



0.3333333333333333


In [10]:
display(Markdown(training_df.code.iloc[1]))

def extract_tools_and_methods(code_string):
    """
    Extract tools and their methods from a code string.

    :param code_string: String containing the code with tool method calls.
    :return: List of tuples where each tuple contains the tool and its method.
    """
    # Regular expression to match tool and method calls
    pattern = re.compile(r"self\.tools\['(\w+)'\]\.(\w+)\(")
    
    # Find all matches
    matches = pattern.findall(code_string)
    
    # Create a list of tuples (tool, method)
    tools_and_methods = [(match[0], match[1]) for match in matches]
    
    return tools_and_methods

extract_tools_and_methods(training_df.code.iloc[1])

```python
# Adding recursion to the root of the learning plan, similar to other algorithm-related subjects.
self.tools['LearningPlanManager'].add_subject(
    subject_name="Recursion", 
    importance=1
)

self.tools['LearningPlanManager'].add_subject(
    subject_name="Recursion Concepts and Common Problems", 
    importance=1,
    parent_subject="Recursion"
)

self.tools['LearningPlanManager'].add_subject(
    subject_name="Tree and Graph Traversal", 
    importance=0.9,
    parent_subject="Recursion"
)

self.tools['LearningPlanManager'].add_subject(
    subject_name="Recursion Complexity Analysis",
    importance=0.5,
    parent_subject="Recursion"
)

self.tools['LearningPlanManager'].add_subject(
    subject_name="Debugging Recursion", 
    importance=0.2,
    parent_subject="Recursion"
)

# Update memory with user's goal
self.tools['MemoryManager'].update_memory('goal', "Improve on algorithms and data structures with special focus on recursion" )

# Responding to the student with an engaging message.
self.tools['Responder'].respond(
    f"Ah, that's a tricky subject! 🔍 I've added a dedicated section on recursion to your learning plan. \n\n"
    f"{self.tools['LearningPlanManager'].get_plan_tree_markdown('Recursion')}\n\n"
    "Would you like to start by exploring Recursion Concepts and Common Problems?"
)
```

[('LearningPlanManager', 'add_subject'),
 ('LearningPlanManager', 'add_subject'),
 ('LearningPlanManager', 'add_subject'),
 ('LearningPlanManager', 'add_subject'),
 ('LearningPlanManager', 'add_subject'),
 ('MemoryManager', 'update_memory'),
 ('Responder', 'respond'),
 ('LearningPlanManager', 'get_plan_tree_markdown')]

In [11]:
def evaluate(gt, pred):
    gt_tools_methods = extract_tools_and_methods(gt)
    pred_tools_methods = extract_tools_and_methods(pred)
    return {
        'jaccard_distance' : jaccard_distance(gt_tools_methods, pred_tools_methods),
        'edit_distance': normalized_list_levenshtein_distance(gt_tools_methods, pred_tools_methods)
    }

# Finetune openAI


In [43]:
def _create_openai_example(system_prompt, user_prompt, code_reponse):
    return {
        "messages": 
        [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
            {"role": "assistant", "content": code_reponse}
        ]
    }

def get_system_prompt(row):
    system_prompt = '''You are an AI Teacher guiding a student toward their goals. Prepare customized plans, track their progress and skills, and personalize your responses using humor, challenges, and appreciation based on their personality ,goal and progress.


        # Instructions:

        - Generate Only Python Code to call the appropriate tools only using self.tools['<toolname>'].method(args).
        - No New Methods/Classes: Only use provided tools/methods.
        - No tools except you will output anything to the user. So you are responsible to pass on the results of tool calls to the user.
        - Memory: Store short but important information in memory which might be needed later.
        - LearningPlan: Focus on locking a customized learning plan to track progress.
        - Confirmation: Confirm with user before changing anything.
        - Responder: User responder to respond to the user. Always respond with a question.
        - Code guidelines: Always end the python code with either a Responder or AITeacher tool call.
        '''
    return re.sub(r'^ {8}', '', system_prompt, flags=re.MULTILINE)

def get_user_prompt(row):
    prompt = f'''
        # Tools Available:
        {row['Tools Available']}

        # Memory: 
        {row['Memory']}

        # Additional Info: 
        {row['Additional Info']}

        # Learning Plan:
        {row['Learning Plan']} 
        
        # Current Time: {row['Current Time']}

        # Conversation History (Last 10): 
        {row['Conversation History (Last 10)']}
        {row['Student Input']}

        # Plan code: 
        '''
    prompt = re.sub(r'^ {8}', '', prompt, flags=re.MULTILINE)
    return prompt

def create_openai_example(row):
    example =  _create_openai_example(
        get_system_prompt(row),
        get_user_prompt(row),
        row['code']
    )
    return json.dumps(example)
    

In [44]:
data = components_df.apply(create_openai_example, axis = 1).values
print(data[0])

{"messages": [{"role": "system", "content": "You are an AI Teacher guiding a student toward their goals. Prepare customized plans, track their progress and skills, and personalize your responses using humor, challenges, and appreciation based on their personality ,goal and progress.\n\n\n# Instructions:\n\n- Generate Only Python Code to call the appropriate tools only using self.tools['<toolname>'].method(args).\n- No New Methods/Classes: Only use provided tools/methods.\n- No tools except you will output anything to the user. So you are responsible to pass on the results of tool calls to the user.\n- Memory: Store short but important information in memory which might be needed later.\n- LearningPlan: Focus on locking a customized learning plan to track progress.\n- Confirmation: Confirm with user before changing anything.\n- Responder: User responder to respond to the user. Always respond with a question.\n- Code guidelines: Always end the python code with either a Responder or AITeac

In [45]:
len(data)

20

In [46]:
OPEN_AI_DATA_JSONL = 'openai_training.jsonl'
with open (OPEN_AI_DATA_JSONL, 'w') as f:
    for ex in data:
        f.write(ex + '\n')

In [47]:
with open (OPEN_AI_DATA_JSONL, 'r') as f:
    d = f.readlines()
    display(json.loads(d[0])['messages'])

[{'role': 'system',
  'content': "You are an AI Teacher guiding a student toward their goals. Prepare customized plans, track their progress and skills, and personalize your responses using humor, challenges, and appreciation based on their personality ,goal and progress.\n\n\n# Instructions:\n\n- Generate Only Python Code to call the appropriate tools only using self.tools['<toolname>'].method(args).\n- No New Methods/Classes: Only use provided tools/methods.\n- No tools except you will output anything to the user. So you are responsible to pass on the results of tool calls to the user.\n- Memory: Store short but important information in memory which might be needed later.\n- LearningPlan: Focus on locking a customized learning plan to track progress.\n- Confirmation: Confirm with user before changing anything.\n- Responder: User responder to respond to the user. Always respond with a question.\n- Code guidelines: Always end the python code with either a Responder or AITeacher tool ca

In [48]:
from openai import OpenAI
client = OpenAI()

client.files.create(
  file=open('openai_training.jsonl', "rb"),
  purpose="fine-tune"
)

FileObject(id='file-gyv7ijxpV1Q1CEeQSw0Z0XDL', bytes=154772, created_at=1725719550, filename='openai_training.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [49]:

client.fine_tuning.jobs.create(
  training_file='file-gyv7ijxpV1Q1CEeQSw0Z0XDL', 
  model="gpt-4o-mini-2024-07-18",
  hyperparameters={
    "n_epochs":2
  }
)

FineTuningJob(id='ftjob-pkgZxglm6zEyxSJYzG3XzlJ4', created_at=1725719594, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-H3gA3aIB39O3UI9PxCnRxxWD', result_files=[], seed=1294795577, status='validating_files', trained_tokens=None, training_file='file-gyv7ijxpV1Q1CEeQSw0Z0XDL', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [56]:
client.fine_tuning.jobs.list(limit=10)


print(client.fine_tuning.jobs.retrieve('ftjob-pkgZxglm6zEyxSJYzG3XzlJ4'))

FineTuningJob(id='ftjob-pkgZxglm6zEyxSJYzG3XzlJ4', created_at=1725719594, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=1725720044, hyperparameters=Hyperparameters(n_epochs=5, batch_size=1, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-H3gA3aIB39O3UI9PxCnRxxWD', result_files=[], seed=1294795577, status='running', trained_tokens=None, training_file='file-gyv7ijxpV1Q1CEeQSw0Z0XDL', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
